In [1]:
from dotenv import load_dotenv
import os
from main import (
    add_metric,
    remove_metric,
    get_metrics,
    validate_user_input,
    export_to_excel,
    create_human_review_template,
    generate_feedback_questions,
)
import pandas as pd

/var/folders/wr/0gwyb5t142l5ys9ymbzy5z380000gn/T/ipykernel_15324/4197826368.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from main import (


In [2]:
env_path = "/Users/ford/Documents/coding/confidential/.env"
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "API key is missing"

In [3]:
data_path = "/Users/ford/Documents/coding/confidential/250327_data_from_RMI_from_productionbuild_Q3-2024_label.xlsx"
# assert is file exist
assert os.path.exists(data_path)

In [4]:
data_df = pd.read_excel(data_path)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1783 entries, 0 to 1782
Data columns (total 82 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   id                                                            789 non-null    float64
 1   control_name                                                  789 non-null    object 
 2   control_owner                                                 789 non-null    object 
 3   have_control                                                  1783 non-null   bool   
 4   control_freq                                                  0 non-null      float64
 5   control_effective_date                                        789 non-null    object 
 6   risk_question_id                                              1783 non-null   int64  
 7   control_des                                                   789 non

In [5]:
selected_df = data_df[data_df["iscustom"] == True]
# column risk_desc unique values
risk_desc_unique_list = selected_df["risk_desc"].unique().tolist()
selected_df.shape

(268, 82)

In [6]:
len(risk_desc_unique_list)

174

In [7]:
risk_desc_unique_list = risk_desc_unique_list[:1]
# risk_desc_unique_list = risk_desc_unique_list[:10]

In [8]:
# Initialize default metrics
add_metric(
    "Have cause",
    "Text identifies and explains the underlying root cause or triggering factors of the risk",
)
add_metric(
    "Have risk event",
    "Text clearly describes what could go wrong or the specific risk scenario that may occur",
)
add_metric(
    "Have impact",
    "Text describes potential consequences or outcomes resulting from the risk event",
)

{'Have cause': 'Text identifies and explains the underlying root cause or triggering factors of the risk',
 'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}

In [9]:
get_metrics()

{'Have cause': 'Text identifies and explains the underlying root cause or triggering factors of the risk',
 'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}

In [10]:
# user_inputs = [
#     "ไม่จัดส่งรายงานพลังงานประจำปี",
#     "ถุงจัมโบ้เสียหายในการจัดเก็บและขนส่ง ทำให้เกิดการปนของสายพันธุ์",
#     "เครื่องจักร Brakedown บ่อย",
# ]
user_inputs = risk_desc_unique_list

In [11]:
all_result = []
for user_input in user_inputs:
    result, feedback_dict = validate_user_input(user_input, export=False)
    all_result.append(result)

Validating user input: 'การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัดซื้อ'
Using provider: openai, model: default
Metrics: {'Have cause': 'Text identifies and explains the underlying root cause or triggering factors of the risk', 'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur', 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}
Validating text on metric: Have cause
Validating text on metric: Have risk event
Validating text on metric: Have impact

Metric: Have cause - INVALID
Reason: The text identifies a specific risk related to fraud in the expense reimbursement and procurement process but does not explain the underlying root cause or triggering factors of this risk. To meet the validation criteria, it should provide insights into why this fraud occurs, such as lack of oversight, inadequate controls, or cultural factors that may contribute to such behavior.

Metric: Have risk e

In [12]:
all_result[:3]

[[{'text': 'การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัดซื้อ',
   'metric': 'Have cause',
   'metric_detail': 'Text identifies and explains the underlying root cause or triggering factors of the risk',
   'additional_information': None,
   'is_valid': False,
   'reason': 'The text identifies a specific risk related to fraud in the expense reimbursement and procurement process but does not explain the underlying root cause or triggering factors of this risk. To meet the validation criteria, it should provide insights into why this fraud occurs, such as lack of oversight, inadequate controls, or cultural factors that may contribute to such behavior.',
   'provider': 'openai',
   'model': 'default'},
  {'text': 'การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัดซื้อ',
   'metric': 'Have risk event',
   'metric_detail': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
   'additional_information': None,
   'is_valid': True,
   'reason': 'The text clea

In [13]:
all_result

[[{'text': 'การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัดซื้อ',
   'metric': 'Have cause',
   'metric_detail': 'Text identifies and explains the underlying root cause or triggering factors of the risk',
   'additional_information': None,
   'is_valid': False,
   'reason': 'The text identifies a specific risk related to fraud in the expense reimbursement and procurement process but does not explain the underlying root cause or triggering factors of this risk. To meet the validation criteria, it should provide insights into why this fraud occurs, such as lack of oversight, inadequate controls, or cultural factors that may contribute to such behavior.',
   'provider': 'openai',
   'model': 'default'},
  {'text': 'การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัดซื้อ',
   'metric': 'Have risk event',
   'metric_detail': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
   'additional_information': None,
   'is_valid': True,
   'reason': 'The text clea

In [14]:
len(all_result)

1

In [15]:
feedback_dict["consolidated_feedback"].keys()

dict_keys(['invalid_metrics', 'question', 'examples', 'original_text'])

In [16]:
feedback_dict["consolidated_feedback"]

{'invalid_metrics': ['Have cause', 'Have impact'],
 'question': 'Your risk description needs improvement in the following areas: Have cause, Have impact. Please provide a more comprehensive description that addresses these aspects.',
 'examples': {'choice1': 'การทุจริตในกระบวนการเบิกจ่ายค่าใช้จ่ายและการจัดซื้อเกิดจากการขาดการตรวจสอบที่เข้มงวด ซึ่งอาจนำไปสู่การใช้จ่ายที่ไม่เหมาะสมและการสูญเสียทรัพยากรทางการเงินที่สำคัญ ส่งผลกระทบต่อความเชื่อมั่นของผู้มีส่วนได้ส่วนเสียและภาพลักษณ์ขององค์กร.',
  'choice2': 'การทุจริตในกระบวนการเบิกจ่ายค่าใช้จ่ายและการจัดซื้อมีสาเหตุมาจากการขาดการควบคุมภายในที่มีประสิทธิภาพ ซึ่งอาจทำให้เกิดการจัดซื้อที่ไม่โปร่งใสและการใช้จ่ายที่ไม่จำเป็น ส่งผลให้เกิดความเสียหายทางการเงินและลดประสิทธิภาพในการดำเนินงานขององค์กร.',
  'choice3': 'การทุจริตในกระบวนการเบิกจ่ายค่าใช้จ่ายและการจัดซื้อเกิดจากการขาดการฝึกอบรมและความตระหนักรู้ในเรื่องจริยธรรมของพนักงาน ซึ่งอาจนำไปสู่การตัดสินใจที่ไม่ถูกต้องและการใช้จ่ายที่ไม่เหมาะสม ส่งผลกระทบต่อความยั่งยืนทางการเงินและความน่าเชื่อถื

In [17]:
feedback_dict["consolidated_feedback"]["examples"]

{'choice1': 'การทุจริตในกระบวนการเบิกจ่ายค่าใช้จ่ายและการจัดซื้อเกิดจากการขาดการตรวจสอบที่เข้มงวด ซึ่งอาจนำไปสู่การใช้จ่ายที่ไม่เหมาะสมและการสูญเสียทรัพยากรทางการเงินที่สำคัญ ส่งผลกระทบต่อความเชื่อมั่นของผู้มีส่วนได้ส่วนเสียและภาพลักษณ์ขององค์กร.',
 'choice2': 'การทุจริตในกระบวนการเบิกจ่ายค่าใช้จ่ายและการจัดซื้อมีสาเหตุมาจากการขาดการควบคุมภายในที่มีประสิทธิภาพ ซึ่งอาจทำให้เกิดการจัดซื้อที่ไม่โปร่งใสและการใช้จ่ายที่ไม่จำเป็น ส่งผลให้เกิดความเสียหายทางการเงินและลดประสิทธิภาพในการดำเนินงานขององค์กร.',
 'choice3': 'การทุจริตในกระบวนการเบิกจ่ายค่าใช้จ่ายและการจัดซื้อเกิดจากการขาดการฝึกอบรมและความตระหนักรู้ในเรื่องจริยธรรมของพนักงาน ซึ่งอาจนำไปสู่การตัดสินใจที่ไม่ถูกต้องและการใช้จ่ายที่ไม่เหมาะสม ส่งผลกระทบต่อความยั่งยืนทางการเงินและความน่าเชื่อถือขององค์กร.'}